# Programme data

This notebook prepares data for the Programme theme page.

In [ ]:
import json
from datetime import date

from utils.paths import SITE
from utils.themes.programme import Programme, ProgrammeSlice

In [ ]:
EVENTS = SITE / 'themes/programme/_data/events'
EVENTS.mkdir(exist_ok=True, parents=True)

## Events data

Read events

In [ ]:
programme_data = ProgrammeSlice(range=(date.min, date.today()))

Create an aggregate by month of the events

In [ ]:
(
    programme_data.events
    .aggregate('month', {
        'Events': ('events', sum),
        'Audience': ('audience', sum),
        # 'Records': (len),
    })
    .convert('month', lambda f: f.isoformat())
    .tocsv(EVENTS / 'total_by_month.csv')
)

Aggregate by Project and by Month, and convert months to columns

In [ ]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'events')
    .recast(key='project_name', variablefield='month', missing=0)
    .tocsv(EVENTS / 'monthly_by_project.csv')
)

Aggregate by Project and by Month, and convert projects to columns

In [ ]:
(
    programme_data.events
    .aggregate(['project_name', 'month'], sum, 'events')
    .recast(key='month', variablefield='project_name', missing=0)
    .tocsv(EVENTS / 'monthly_breakdown.csv')
)

## Project summaries

Create a project breakdown

In [ ]:
with open(EVENTS / 'by_project.json', 'w') as f:
    json.dump(
        dict(
            programme_data.project_details.records()
        ),
        f,
        indent=2,
    )

Create a summary file

In [ ]:
programme_data.excluded_events_data

In [ ]:
with open(EVENTS / 'summary.json', 'w') as f:
    json.dump(
        {
            'total': {
                'events': sum(programme_data.events.values('events')),
                'audience': sum(a for a in programme_data.events.values('audience') if a is not None)
            },
            'excluded': dict(programme_data.excluded_events_data.aggregate('validation', len).records()),
            'date': {
                'earliest': min(programme_data.events_data.values('start_date')).isoformat(),
                'latest': max(programme_data.events_data.values('end_date')).isoformat(),
            }
        },
        f,
        indent=2,
    )

## Venues

In [ ]:
education_settings = (
    Programme.venues
    .selectcontains('Org/Venue Type', 'Education Setting')
)

In [ ]:
loading_bay = (
    Programme.venues
    .selectcontains('Organisation &/or Venue Name', 'Loading Bay')
    .addfield('event_report_count', lambda r: len(r['Event Reports']))
    .aggregate(['Organisation &/or Venue Name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
loading_bay

In [ ]:
beacon = (
    Programme.venues
    .selectcontains('Organisation &/or Venue Name', 'Beacon - ')
    .addfield('event_report_count', lambda r: len(r['Event Reports'] or []))
    .aggregate(['Organisation &/or Venue Name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
beacon

In [ ]:
json.dump(
    {
        'loading_bay': dict(loading_bay.records()),
        'beacon': dict(beacon.records()),
    },
    open(EVENTS / 'by_venue.json', 'w')
)